In [1]:
import pandas as pd
import numpy as np
from pyplink import PyPlink
import statsmodels.api as sm
from IPython.display import clear_output

In [2]:
expression = pd.read_table("GD462.GeneQuantRPKM.50FN.samplename.resk10.txt.gz", delim_whitespace=True)
expression.set_index('TargetID', inplace=True)
expression.index = expression.index.str.replace("\.\d+", "", regex=True)
annotation = pd.read_csv("gene_annot.txt", sep="\t")
annotation.set_index('SYM', inplace=True)

In [3]:
expression.shape[0]

23722

In [7]:
#'ENSG00000183696' in annotation.index

True

In [8]:
#annotation

,ID,CHR,START,STOP,TYPE
SYM,,,,,
ENSG00000223972,DDX11L1,1,11868,13052,transcribed_unprocessed_pseudogene
ENSG00000186092,OR4F5,1,69090,70008,protein_coding
ENSG00000177757,FAM87B,1,817370,819834,lincRNA
ENSG00000225880,LINC00115,1,826205,827522,lincRNA
ENSG00000228794,LINC01128,1,827607,859446,processed_transcript
...,...,...,...,...,...
ENSG00000100299,ARSA,22,50622753,50628179,protein_coding
ENSG00000251322,SHANK3,22,50674414,50733298,protein_coding
ENSG00000100312,ACR,22,50738195,50745334,protein_coding


In [9]:
#expression

,Gene_Symbol,Chr,Coord,HG00096,HG00097,HG00099,HG00100,HG00101,HG00102,HG00103,...,NA20810,NA20811,NA20812,NA20813,NA20814,NA20815,NA20816,NA20819,NA20826,NA20828
TargetID,,,,,,,,,,,,,,,,,,,,,
ENSG00000152931,ENSG00000152931.6,5,59783540,0.101858,0.078110,0.048981,0.118597,0.004035,0.010925,-0.000901,...,0.088601,0.240010,0.137175,0.148494,0.038643,0.088509,0.029204,0.024423,0.044816,0.139186
ENSG00000183696,ENSG00000183696.9,7,48128225,8.183805,5.686911,2.434653,3.830894,6.612288,4.709646,7.348876,...,13.428205,6.094500,12.536000,2.217262,3.573394,7.583364,4.052882,1.570378,4.900372,6.737308
ENSG00000139269,ENSG00000139269.2,12,57846106,1.199910,1.573572,0.521616,1.447225,3.565791,1.982681,0.675305,...,3.225880,1.996067,2.854923,2.267343,1.331201,2.187895,1.004250,3.003316,1.984362,1.684954
ENSG00000169129,ENSG00000169129.8,10,116164515,0.831940,0.069778,0.931086,0.620941,1.660668,0.570481,1.259393,...,1.023381,1.127852,0.774409,1.495854,0.895342,1.513521,0.826377,1.021201,0.952502,0.740565
ENSG00000134602,ENSG00000134602.11,X,131157293,27.646422,24.395572,16.445374,24.806650,25.113349,19.233988,27.881116,...,25.079490,28.725528,24.450520,27.264069,26.912814,29.509210,26.462331,25.624009,25.707741,22.824957
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000235472,ENSG00000235472.1,13,29172970,31.582832,34.071123,19.394365,37.523721,33.430473,37.844966,35.843048,...,47.905182,33.224977,39.827675,27.096811,34.686342,37.386766,30.598840,33.516674,32.017940,38.341888
ENSG00000114423,ENSG00000114423.14,3,105588396,14.054749,14.477899,11.584425,12.637956,12.015089,13.750655,15.690696,...,11.723462,9.900372,10.473115,13.433413,15.832594,19.216176,10.213739,14.563192,15.637732,8.357117
ENSG00000243312,ENSG00000243312.2,4,87791344,1.112114,0.831797,0.253228,0.271568,0.486086,1.362640,1.543528,...,1.168991,0.645389,0.819469,0.515448,0.463054,1.580658,0.701396,0.771233,0.857330,0.703369


In [4]:
bim = PyPlink("LDREF/1000G.EUR.1").get_bim()
#bim

,chrom,pos,cm,a1,a2
snp,,,,,
rs3094315,1,752566,0.488776,G,A
rs3131972,1,752721,0.488868,A,G
rs3131969,1,754182,0.489734,A,G
rs1048488,1,760912,0.492507,C,T
rs3115850,1,761147,0.492605,T,C
...,...,...,...,...,...
rs4926502,1,249209140,293.389210,A,G
rs6704311,1,249210707,293.390550,A,G
rs34013644,1,249211350,293.391090,C,T


In [11]:
#np.savetxt("individuals_with_expression.txt", np.array([expression.columns[4:].values, expression.columns[4:].values]).T, fmt='%s')

In [12]:
#for i in range(1,23):
#    ! ~/Downloads/180A/plink2 -bfile /Users/johndriscoll/Downloads/180A/LDREF/1000G.EUR.{i} -keep individuals_with_expression.txt -make-bed -out /Users/johndriscoll/Downloads/180B/LDREF/1000G.EUR.{i}

In [5]:
pyplinkdict = {i: PyPlink("LDREF/1000G.EUR.{}".format(i)) for i in range(1,23)}

In [6]:
pyplinkdict

{1: PyPlink(343 samples; 98,642 markers),
 2: PyPlink(343 samples; 99,735 markers),
 3: PyPlink(343 samples; 83,036 markers),
 4: PyPlink(343 samples; 74,924 markers),
 5: PyPlink(343 samples; 75,164 markers),
 6: PyPlink(343 samples; 75,358 markers),
 7: PyPlink(343 samples; 66,171 markers),
 8: PyPlink(343 samples; 64,975 markers),
 9: PyPlink(343 samples; 55,464 markers),
 10: PyPlink(343 samples; 64,067 markers),
 11: PyPlink(343 samples; 60,977 markers),
 12: PyPlink(343 samples; 58,543 markers),
 13: PyPlink(343 samples; 45,546 markers),
 14: PyPlink(343 samples; 39,484 markers),
 15: PyPlink(343 samples; 35,839 markers),
 16: PyPlink(343 samples; 36,526 markers),
 17: PyPlink(343 samples; 32,218 markers),
 18: PyPlink(343 samples; 35,513 markers),
 19: PyPlink(343 samples; 22,509 markers),
 20: PyPlink(343 samples; 31,101 markers),
 21: PyPlink(343 samples; 17,040 markers),
 22: PyPlink(343 samples; 17,489 markers)}

In [19]:
#(pyplinkdict[1].get_fam()['fid'].values) == (pyplinkdict[2].get_fam()['fid'].values)

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,

In [15]:
#expression[pyplinkdict[1].get_fam()['fid'].values]

,HG00097,HG00099,HG00101,HG00102,HG00103,HG00105,HG00108,HG00109,HG00110,HG00111,...,NA20809,NA20810,NA20811,NA20812,NA20813,NA20814,NA20815,NA20819,NA20826,NA20828
TargetID,,,,,,,,,,,,,,,,,,,,,
ENSG00000152931,0.078110,0.048981,0.004035,0.010925,-0.000901,0.098863,0.076937,0.246899,0.088554,0.051973,...,0.015805,0.088601,0.240010,0.137175,0.148494,0.038643,0.088509,0.024423,0.044816,0.139186
ENSG00000183696,5.686911,2.434653,6.612288,4.709646,7.348876,8.721889,4.090099,6.183830,8.721428,5.029112,...,5.311783,13.428205,6.094500,12.536000,2.217262,3.573394,7.583364,1.570378,4.900372,6.737308
ENSG00000139269,1.573572,0.521616,3.565791,1.982681,0.675305,2.561376,1.269440,2.069078,2.551473,2.674330,...,2.284592,3.225880,1.996067,2.854923,2.267343,1.331201,2.187895,3.003316,1.984362,1.684954
ENSG00000169129,0.069778,0.931086,1.660668,0.570481,1.259393,1.479124,1.010198,0.559784,0.477960,0.491458,...,0.229538,1.023381,1.127852,0.774409,1.495854,0.895342,1.513521,1.021201,0.952502,0.740565
ENSG00000134602,24.395572,16.445374,25.113349,19.233988,27.881116,28.579857,25.384714,22.851098,26.239197,22.157278,...,25.412772,25.079490,28.725528,24.450520,27.264069,26.912814,29.509210,25.624009,25.707741,22.824957
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000235472,34.071123,19.394365,33.430473,37.844966,35.843048,32.287466,29.742259,39.869214,34.008313,40.068172,...,31.991946,47.905182,33.224977,39.827675,27.096811,34.686342,37.386766,33.516674,32.017940,38.341888
ENSG00000114423,14.477899,11.584425,12.015089,13.750655,15.690696,15.461657,9.870305,13.327785,10.335004,14.883169,...,12.386452,11.723462,9.900372,10.473115,13.433413,15.832594,19.216176,14.563192,15.637732,8.357117
ENSG00000243312,0.831797,0.253228,0.486086,1.362640,1.543528,0.900239,0.755062,0.949275,0.571035,0.242975,...,0.240227,1.168991,0.645389,0.819469,0.515448,0.463054,1.580658,0.771233,0.857330,0.703369


In [8]:
#subset expression data for those we have genomes for
expression = expression[pyplinkdict[1].get_fam()['fid'].values]

In [12]:
symbol_list = []
for symbol in expression.index:
    if symbol in annotation.index:
        symbol_list += [symbol]
np.array(symbol_list).T.tofile("eQTL_symbol_list_chip.csv")

In [23]:

#expression = expression[pyplinkdict[curr_chromosome].get_fam()['fid'].values]
progress = 0
for symbol in expression.index:

    total_genes = len(expression.index)
    progress += 1

    if symbol not in annotation.index:
        continue
    gene_name, chromosome, gene_start = annotation.loc[symbol][["ID", "CHR", "START"]].values
    cis_start, cis_end = gene_start - 500000, gene_start + 500000
    if cis_start < 0:
        cis_start = 0
    max_stop = max(annotation["STOP"])
    if cis_end > max_stop:
        cis_end = max_stop

    curr_bim = pyplinkdict[chromosome].get_bim()
    curr_expression = expression.loc[symbol].values

    #select snps within gene body
    snp_subset = curr_bim[(curr_bim['pos'] > cis_start) & (curr_bim['pos'] < cis_end)].reset_index()

    #display(snp_subset)
    summary_matrix = []

    N = pyplinkdict[chromosome].get_nb_samples()
    snp_subset['N'] = [N] * snp_subset.shape[0]
    snp_subset['type'] = ['cc'] * snp_subset.shape[0]

    genotypes = []
    mafs = []
    for _, [snp, a1] in snp_subset[['snp', 'a1']].iterrows():
        #print(snp)
        alleles = pyplinkdict[chromosome].get_geno_marker(snp)
        unique_cts = np.unique(alleles, return_counts=True)[1]
        maf = min(unique_cts/sum(unique_cts))
        genotypes += [alleles]
        mafs += [maf]
    snp_subset['genotypes'] = genotypes
    snp_subset['MAF'] = mafs

    #print(len(curr_expression), len(genotypes[0]))
    snp_subset['results'] = snp_subset['genotypes'].apply(lambda x: sm.OLS(curr_expression, x).fit())

    snp_subset['pval'] = snp_subset['results'].apply(lambda x: x.pvalues[0])
    snp_subset['bse'] = snp_subset['results'].apply(lambda x: x.bse[0])
    snp_subset['varbeta'] = snp_subset['bse']**2
    snp_subset['effect_size'] = np.e**snp_subset['results'].apply(lambda x: x.params[0])
    snp_subset['beta'] = snp_subset['results'].apply(lambda x: x.params[0])

    snp_subset[['beta', 'varbeta', 'snp', 'pos', 'type', 'N', 'MAF', 'pval']].to_csv("eQTL_subsets/{}.csv".format(symbol), index=False)
    #print("{}".format(symbol))
    clear_output(wait=True)
    display("Wrote {}/{} summary stats".format(progress, total_genes, curr_chromosome))


'Wrote 58/23722 summary stats'

KeyboardInterrupt: 

In [18]:
len(expression.index)

23722